<a href="https://colab.research.google.com/github/agrawalsourabh/DeepLearning/blob/master/Human_or_Horse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human - or - Horse</br>
Built a model that classify the image as Horse image or Human image.
</br>

**Learning Objective:**
* Download the data to local drive.
* Configure a neural network model
* Built the model.
* Tweak the hyperparameters to increase accuracy.
* Configure other NN model.
* Deploy the model to andorid app.

In [0]:
# Download the public image dataset

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

**Import Libraries**

In [0]:
import os
import zipfile

local_zip = "/tmp/horse-or-human.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [0]:
# Directory with our training horse picture
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human picture
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

Let's see how the file names look like.

In [0]:
print("Horse images - ")
train_horse_image = os.listdir(train_horse_dir)
print(train_horse_image[:10])

print("Human images - ")
train_human_image = os.listdir(train_human_dir)
print(train_human_image[:10])




Check total training horse and human images

In [0]:
print('Total human images: ', len(os.listdir(train_human_dir)))
print('Total horse images: ', len(os.listdir(train_horse_dir)))

**Visualise training data**

In [0]:
# import libraries

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows = 4
ncols = 4

# Setup plot size
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)


# Horse Images
sbplt_index = 0
for horse_fname in train_horse_image[:4] :
  
  sbplt_index = sbplt_index + 1
  
  horse_fileName = os.path.join(train_horse_dir, horse_fname)
  
  sp = plt.subplot(nrows, ncols, sbplt_index)
  sp.axis('Off')
  
  horse_img = mpimg.imread(horse_fileName)
  plt.imshow(horse_img)
  
  
  
plt.show()


# Setup plot size
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

# Human Images
sbplt_index = 0
for human_fname in train_human_image[:4] :
  sbplt_index = sbplt_index +1
  human_fileName = os.path.join(train_human_dir, human_fname)
  
  sp = plt.subplot(nrows, ncols, sbplt_index)
  sp.axis('Off')
  
  human_img = mpimg.imread(human_fileName)
  plt.imshow(human_img)
  
plt.show()
  


**Build a model from Scratch**

In [0]:
import tensorflow as tf

**Configure a model**
* Sequence
* Conv - 1st layer
* Max pooling
* Conv - 2nd layer
* Max pooling
* Flatten
* Fully Connected layer
* Output layer

In [0]:
model = tf.keras.models.Sequential()

# 1st Conv layer
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), 
                                 activation='relu', input_shape=(300, 300, 3)))
# Max pooling
model.add(tf.keras.layers.MaxPooling2D(2, 2))


# 2nd Conv layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), 
                                 activation='relu'))
# Max pooling
model.add(tf.keras.layers.MaxPooling2D(2, 2))

# Flatten
model.add(tf.keras.layers.Flatten())

# Hidden layer - with 512 neurons
model.add(tf.keras.layers.Dense(512, activation="relu"))

# Output layer - with only one neuron
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))



model.summary() prints the summary of the model

In [0]:
print(model.summary())

**Complile the model**

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

**Data Preprocessing**
We'll use data generator form Keras.Preprocessing

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# all images will be rescaled by /255
train_datagen = ImageDataGenerator(rescale = 1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

**Training Model**
Let's train the model for 10 epochs

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

**Predict using local image**

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 